In [24]:
import os
import gradio as gr 

from constants import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] =  OPENAI_API_KEY

from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAI


In [25]:
template_string = """Generate a real estate description
of a house, it has to be maximum 100 words long and have 
the following characteristics:

Neighborhood: {neighborhood}\n
Price: {price}\n sqft
Bedrooms: {bedrooms}\n
Bathrooms: {bathrooms}\n
House Size: {size}\n


Here are characteristics about the neighborhood:
{neighborhood_details}

And here some house specific details:
{house_specific_details}

It's mandatory to include these exact information in the description.
Try not to start the description with the words "Welcome to".

"""

extracted_text = """
Take a look at the following example to have an idea of
the expected output, the text is delimited by triple backticks:

```{example}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [26]:
prompt_template.messages[0].input_variables

['bathrooms',
 'bedrooms',
 'house_specific_details',
 'neighborhood',
 'neighborhood_details',
 'price',
 'size']

In [48]:
completion_model_name = "gpt-3.5-turbo-instruct"


bot = OpenAI(
    model_name=completion_model_name, 
    temperature=0.5, 
)

def generate_listings(
        INPUT_NEIGHBORHOOD, 
        INPUT_PRICE, 
        INPUT_BEDROOMS, 
        INPUT_BATHROOMS, 
        INPUT_SIZE, 
        neighborhood_residential_location_details, 
        neighborhood_additional_details, 
        house_type_details, 
        house_style_details,
        house_additional_details 
        ):

    neighborhood_additional_details.append(neighborhood_additional_details)
    NEIGHBORHOOD_DETAILS = "\n".join(neighborhood_residential_location_details)

    house_additional_details.append(house_type_details)
    house_additional_details.append(house_style_details)
    HOUSE_DETAILS = "\n".join(house_additional_details)

    prompt_description = prompt_template.format_messages(
        neighborhood = INPUT_NEIGHBORHOOD, 
        price = INPUT_PRICE, 
        bedrooms = INPUT_BEDROOMS, 
        bathrooms= INPUT_BATHROOMS, 
        size = INPUT_SIZE,  
        neighborhood_details = NEIGHBORHOOD_DETAILS,
        house_specific_details = HOUSE_DETAILS,

    )

    bot_description = bot.invoke(prompt_description)


    return bot_description




In [49]:


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1):
            INPUT_NEIGHBORHOOD = gr.Dropdown(
                    ["Manhattan", "Brooklyn", "Queens", "The Bronx", "Staten Island"], 
                    #value=["swam", "slept"], 
                    multiselect=False, 
                    label="Borough", 
                    info="Select NYC's Borough"
            )

            INPUT_BATHROOMS = gr.Dropdown(
                    ["1", "2", "3", "4", "5"], 
                    # value=["1"], 
                    multiselect=False, 
                    label="Bathrooms", 
                    info="Select number of bathrooms."
            )

            INPUT_BEDROOMS = gr.Dropdown(
                    ["1", "2", "3", "4", "5"], 
                    # value=["1"], 
                    multiselect=False, 
                    label="Bedrooms", 
                    info="Select number of bedrooms."
            )


            INPUT_SIZE = gr.Slider(
                800, 
                10_000, 
                value=800, 
                label="Size in SQFT", 
                info="Introduce the size of your house:"
            )

            INPUT_PRICE = gr.Slider(
                60_000, 
                1_000_000, 
                value=1_000, 
                label="Price in USD", 
                info="Introduce the select the home's price:"
            )


            neighborhood_residential_location_details = gr.Dropdown(
                [
                    "Quiet residential location", 
                    "Bustling residential location", 
                    "Moderately quiet residential location", 
                ], 
                value=[], 
                multiselect=False, 
                label="Residential Location Details", 
                info="Describe the location environment."
            )

            neighborhood_additional_details = gr.Dropdown(
                [
                    "Secure neighborhood",
                    "Close to touristic places", 
                    "Nice restaurants in the area", 
                    "Good bars in the area", 
                    "Fantastic disco bars in the area", 
                    "Easiness to get public transportation",
                    "Bike-friendly roads",
                    "Closeness to schools", 
                    "Closeness to work", 
                    "Closeness to malls", 
                    "Closeness to supermarkets",
                    "Closeness to drug stores",  
                ], 
                value=[], 
                multiselect=True, 
                label="Neighborhood Details", 
                info="Introduce Neighborhood details"
            )
            # neighborhood_details = neighborhood_residential_location_details + neighborhood_additional_details

            # neighborhood_additional_details = gr.Textbox(
            #     value="", 
            #     label="Neighborhood Details", 
            #     info="This is an optional space that you can use to add more neighborhood details."
            # )

            # NEIGHBORHOOD_DETAILS = "\n".join(neighborhood_details)


            house_type_details = gr.Dropdown(
                [
                    "Single-family home",
                    "Condo", 
                    "Townhouse", 
                ], 
                value=[], 
                multiselect=False, 
                label="House Type ", 
                info="Introduce House Type"
            )

            house_style_details = gr.Dropdown(
                [
                    "Classic style", 
                    "Contemporary style", 
                    "Minimalist style"
                ], 
                value=[], 
                multiselect=False, 
                label="House Style Details", 
                info="Introduce Style:"
            )

            house_additional_details = gr.Dropdown(
                [
                    "Solar energy", 
                    "Generous parking",
                    "Gardens", 
                    "Pool", 
                    "Reduce mobility friendly", 
                    "Pet friendly", 
                    "Requires remodelling", 
                ], 
                value=[], 
                multiselect=True, 
                label="House Specific Details", 
                info="Introduce Specific Details"
            )

            # house_specific_details = house_type_details + house_style_details + house_additional_details

            # HOUSE_DETAILS = "\n".join(house_specific_details)

            # house_specific_additional_details = gr.Textbox(
            #     value="", 
            #     label="House Specific Details", 
            #     info="This is an optional space that you can use to add more house specific details."
            # )




        with gr.Column(scale=1):
            listing_description = gr.Textbox(value="", label="Output")



    btn = gr.Button(value="Generate Listing!")
    btn.click(
        generate_listings, 
        inputs=[
            INPUT_NEIGHBORHOOD, 
            INPUT_PRICE, 
            INPUT_BEDROOMS, 
            INPUT_BATHROOMS, 
            INPUT_SIZE, 
            neighborhood_residential_location_details, 
            neighborhood_additional_details, 
            house_type_details, 
            house_style_details,
            house_additional_details 
            
        ], 
        outputs=[listing_description])




if __name__ == "__main__":
    demo.launch()

/Users/arturopolancolozano/opt/anaconda3/lib/python3.9/site-packages/gradio/components/dropdown.py:231: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: [] or set allow_custom_value=True.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.
